# SMART-G validation of object surface roughness

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

## Initialisation

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import RoughSurface
from smartg.atmosphere import AtmAFGL
# from smartg.tools.luts import LUT, MLUT, Idx, merge, read_mlut
from luts.luts import Idx
from smartg.tools.smartg_view import smartg_view
# from ipywidgets import interact, interact_manual
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))

import smartg.geometry
from smartg.geometry import Point
import smartg.visualizegeo
from smartg.visualizegeo import receiver_view, cat_view, Mirror, Plane, Transformation, \
    Entity, Analyse_create_entity, Matte, generateHfP, generateHfA
from smartg.smartg import CusForward
import warnings
warnings.filterwarnings("ignore")

## Cox and Munk

This part is a copy from paper-smartg-figure-2018

In [ ]:
from scipy.special import erfc
def Fresnel(thetai, NW=1.33):
    thetat = np.arcsin(sin(thetai)/NW)
    rper   = (NW*np.cos(thetai)-np.cos(thetat))/(NW*np.cos(thetai)+np.cos(thetat))
    rpar   = (np.cos(thetai)-NW*np.cos(thetat))/(np.cos(thetai)+NW*np.cos(thetat))
    return rper**2, rpar**2

def P(beta, WS=2.):
    sig = sqrt(0.003 + 0.00512 *WS)
    return exp(-np.tan(beta)**2/(sig**2))/(np.pi*sig**2)

def Shadow(mus, muv, WS=2.):
    sigp = sqrt(0.5)*sqrt(0.003 + 0.00512 *WS)
    nus = 1/np.tan(np.arccos(mus))/sigp/sqrt(2.)
    lambdaS = (np.exp(-nus**2) -nus*sqrt(np.pi)*erfc(nus))/(2*nus*sqrt(np.pi))
    nuv = 1/np.tan(np.arccos(muv))/sigp/sqrt(2.)
    lambdaV = (np.exp(-nuv**2) -nuv*sqrt(np.pi)*erfc(nuv))/(2*nuv*sqrt(np.pi))
    return 1./(1.+lambdaS+lambdaV)

def LambdaM(avz, sig2) :
    #Mischenko implementation
    if (avz == 1.) : l = 0.;
    else :
        s1 = np.sqrt(np.float64(2)*sig2/np.pi);
        s3 = np.float64(1)/np.sqrt(np.float64(2)*sig2);
        s2 = s3*s3;
        xi = avz;
        xxi=xi*xi;
        dcot =  xi / np.sqrt(np.float64(1)-xxi);
        t1 = np.exp(-dcot*dcot*s2);
        t2 = erfc(dcot*s3);
        l  = 0.5*(s1*t1/dcot-t2);
    return l

def Shadow2(mus, muv, WS=2.):
    sigp = sqrt(0.5)*sqrt(0.003 + 0.00512 *WS)
    return 1./(1.+LambdaM(mus,sigp*sigp)+LambdaM(muv,sigp*sigp))

In [ ]:
def Glint(thetas, thetav, dphi, WS=2., SHADOW=False):
    mus = cos(thetas*np.pi/180.)
    muv = cos(thetav*np.pi/180.)
    shadow = Shadow(mus,muv,WS=WS) if SHADOW else 1.
    cosp= cos(dphi*np.pi/180.)
    mu  = mus*muv + sqrt(1-mus**2)*sqrt(1-muv**2)*cosp
    mub = (mus+muv)/(sqrt(2.+2*mu))
    rho = np.pi*P(np.arccos(mub),WS=WS)/(4*mus*muv*mub**4) * shadow
    rper, rpar= Fresnel(np.arccos(mu)/2.)
    return rper*rho, rpar*rho

In [ ]:
%%time
Dphi=180.

NBPHOTONS=1e5
S=Smartg(double=True, back=False)
th = np.concatenate((np.linspace(0.,80.,num=30), np.array([82.,84.,86.,88., 89.])),axis=0)
fig,ax = subplots(2,3)
ws_l=[2.,15.]
color_l=['r','b']
##########
le={'th_deg':th}
le.update(phi_deg=np.array([Dphi]))
for it, (th0, line) in enumerate(zip([25., 70., 80.],['-','-','-'])):
    for WS, color in zip(ws_l, color_l):
        for shadow, sym in zip([True,False],['.','+']):
            col = it
            surf=RoughSurface(SUR=1,NH2O=1.33, WIND=WS, WAVE_SHADOW=shadow, SINGLE=True)
            m=S.run(THVDEG=th0, NBPHOTONS=NBPHOTONS, surf=surf, wl=443., le=le, water=None, OUTPUT_LAYERS=3)
            ax[0,col].plot(th,abs(m['I_up (0+)'].sub()[Idx(Dphi),:].data), sym, color=color)

            Iper, Ipar = Glint(th0, th, Dphi, WS=WS, SHADOW=shadow)
            ax[0,col].plot(th, abs(0.5*(Iper+Ipar)), line, color=color)            
            ax[0,col].set_title(r'$\theta_s$:%.0f'%th0)
            if it==0:                
                ax[0,col].set_ylim([0.001,2])
                label=''
            else : 
                if shadow:
                    label = r'$w_s$= {0:.0f}m/s + wave shadows'.format(WS)
                else :
                    label = r'$w_s$= {0:.0f}m/s'.format(WS)
                ax[0,col].set_ylim([0.001,100])
            ax[1,col].plot(th,100* (abs(m['I_up (TOA)'].sub()[Idx(Dphi),:].data) - abs(0.5*(Iper+Ipar)))/abs(0.5*(Iper+Ipar)), 
                           sym+color, label=label)
            ax[1,col].set_ylim([-1,1])
            ax[1,col].set_xlabel(r'$\theta_v$',fontsize=14)
        ax[1,0].set_ylabel(r'$\Delta (\%)$',fontsize=14)
        ax[0,0].set_ylabel(r'$\rho_g$',fontsize=14)

sca(ax[0,2])
plot([0,6],[90,90],'.'+color_l[0])
text(8,90,r'$w_s$= {0:.0f}m/s + wave shadows'.format(ws_l[0]), color=color_l[0])
plot([0,6],[80,80],'+'+color_l[0])
text(8,80,r'$w_s$= {0:.0f}m/s'.format(ws_l[0]), color=color_l[0])
plot([0,6],[70,70],'.'+color_l[1])
text(8,70,r'$w_s$= {0:.0f}m/s + wave shadows'.format(ws_l[1]), color=color_l[1])
plot([0,6],[60,60],'+'+color_l[1])
text(8,60,r'$w_s$= {0:.0f}m/s'.format(ws_l[1]), color=color_l[1])
plot([0,6],[50,50],'-k')
text(8,50,'BRDF', color='k')
fig.set_size_inches(12,8)
#fig.savefig('/home/did/RTC/SMART-G/notebooks/Validation_paper_smartg/2018_v2/glitter_single.png', dpi=600)
#fig.savefig('/home/did/RTC/SMART-G/notebooks/Validation_paper_smartg/2018_v2/glitter.png', dpi=600)

In [ ]:
# le={'th_deg':th}
# Dphi = np.linspace(0,360, num=37)
# le.update(phi_deg=np.array([Dphi]))
surf=RoughSurface(SUR=1,NH2O=1.33, WIND=2, WAVE_SHADOW=False, SINGLE=True)
m=S.run(THVDEG=25., NBPHOTONS=NBPHOTONS*1000, atm = AtmAFGL('afglms', tauR=1e-19), NBPHI=90, NBTHETA=900,
        surf=surf, wl=443., le=None, water=None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m, QU=True, ind=Idx([0, 10]))

## Replication of Cox and Munk with a plane object at BOA

In [ ]:
# Size of the plane (doesn't really matter since all rays target the point x=0,y=0,z=0)
wMx = 1
wMy = 1

#relative refractive index air/water
NH2O = 1.33

# Parameter(alpha) for the slope error according to Cox and Munk, function of the speed of wind
WINDSPEED02=2
alpha02 = sqrt(0.003 + 0.00512*WINDSPEED02)

# Distribution used for the microfacet model
dist = "Beckmann"
#dist = "GGX" # alpha GGX need to be modified compared to alpha beckmann

# The plane created as a Mirror with roughness = alpha
Mir02 = Entity(name = "reflector", \
               materialAV = Mirror(reflectivity = 1., roughness = alpha02, shadow = False,\
                                   nind = NH2O, distribution = dist), \
               materialAR = Matte(), \
               geo = Plane( p1 = Point(-wMx, -wMy, 0.),
                            p2 = Point(wMx, -wMy, 0.),
                            p3 = Point(-wMx, wMy, 0.),
                            p4 = Point(wMx, wMy, 0.) ), \
               transformation = Transformation( rotation = np.array([0., 0., 0.]), \
                                                translation = np.array([0., 0., 0.]) ))

# With consideration of Shadowing-Masking effect
Mir02S = Entity(Mir02)
Mir02S.materialAV = Mirror(reflectivity = 1., roughness = alpha02, shadow = True, nind = NH2O, \
                           distribution = dist)

# The same but with a wind speed equal 15 m/s
WINDSPEED15=15
alpha15 = sqrt(0.003 + 0.00512*WINDSPEED15)

Mir15 = Entity(Mir02); Mir15S = Entity(Mir02);
Mir15.materialAV = Mirror(reflectivity = 1., roughness = alpha15, shadow = False, nind = NH2O, 
                          distribution = dist)
Mir15S.materialAV = Mirror(reflectivity = 1., roughness = alpha15, shadow = True, nind = NH2O, \
                           distribution = dist)

### Solar zenith angle = 25 degrees

In [ ]:
m1 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=25., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m1, QU=True, ind=Idx([0, 10])) # 2 m/s

In [ ]:
m2 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=25., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m2, QU=True, ind=Idx([0, 10])) # 2 m/s + shadow

In [ ]:
m3 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=25., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir15], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m3, QU=True, ind=Idx([0, 10])) # 15 m/s

In [ ]:
m4 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=25., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 1,
    myObjects=[Mir15S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m4, QU=True, ind=Idx([0, 10])) # 15 m/s + shadow

### Solar zenith angle = 70 degrees

In [ ]:
m5 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=70., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m5, QU=True, ind=Idx([0, 10])) # 2 m/s

In [ ]:
m6 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=70., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m6, QU=True, ind=Idx([0, 10])) # 2 m/s + shadow

In [ ]:
m7 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=70., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir15], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m7, QU=True, ind=Idx([0, 10])) # 15 m/s

In [ ]:
m8 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=70., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir15S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m8, QU=True, ind=Idx([0, 10])) # 15 m/s + shadow

### Solar zenith angle = 80 degrees

In [ ]:
m9 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=80., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m9, QU=True, ind=Idx([0, 10])) # 2 m/s

In [ ]:
m10 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=80., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir02S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m10, QU=True, ind=Idx([0, 10])) # 2 m/s + shadow

In [ ]:
m11 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=80., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir15], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m11, QU=True, ind=Idx([0, 10])) # 15 m/s

In [ ]:
m12 = Smartg(double = True, obj3D = True).run( surf = None, atm = AtmAFGL('afglms', tauR=1e-19),
    THVDEG=80., wl=443., NBPHOTONS=1e8, NBPHI=90, NBTHETA=900, IsAtm = 0,
    myObjects=[Mir15S], water = None, OUTPUT_LAYERS=3)

In [ ]:
_=smartg_view(m11, QU=True, ind=Idx([0, 10])) # 15 m/s + shadow